In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb  # LightGBM
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")


In [33]:
def preprocessing_expected(df, typ):
    df['prob_0'] = 100 - df['rainfall_' + typ + '.v01']
    df['prob_1'] = df['rainfall_' + typ + '.v01'] - df['rainfall_' + typ + '.v02']
    df['prob_2'] = df['rainfall_' + typ + '.v02'] - df['rainfall_' + typ + '.v03']
    df['prob_3'] = df['rainfall_' + typ + '.v03'] - df['rainfall_' + typ + '.v04']
    df['prob_4'] = df['rainfall_' + typ + '.v04'] - df['rainfall_' + typ + '.v05']
    df['prob_5'] = df['rainfall_' + typ + '.v05'] - df['rainfall_' + typ + '.v06']
    df['prob_6'] = df['rainfall_' + typ + '.v06'] - df['rainfall_' + typ + '.v07']
    df['prob_7'] = df['rainfall_' + typ + '.v07'] - df['rainfall_' + typ + '.v08']
    df['prob_8'] = df['rainfall_' + typ + '.v08'] - df['rainfall_' + typ + '.v09']
    df['prob_9'] = df['rainfall_' + typ + '.v09']
    df.drop(columns=[
                    'rainfall_' + typ + '.v01',
                    'rainfall_' + typ + '.v02',
                    'rainfall_' + typ + '.v03',
                    'rainfall_' + typ + '.v04',
                    'rainfall_' + typ + '.v05',
                    'rainfall_' + typ + '.v06',
                    'rainfall_' + typ + '.v07',
                    'rainfall_' + typ + '.v08',
                    'rainfall_' + typ + '.v09'], inplace=True)
    if typ == 'train':
        df = df[df['rainfall_' + typ + '.vv'] >= 0]
    df["expected"] = (df["prob_0"] * 0\
                    +df["prob_1"] * 1\
                    +df["prob_2"] * 2\
                    +df["prob_3"] * 3\
                    +df["prob_4"] * 4\
                    +df["prob_5"] * 5\
                    +df["prob_6"] * 6\
                    +df["prob_7"] * 7\
                    +df["prob_8"] * 8\
                    +df["prob_9"] * 9) / 100
    columns=["prob_0","prob_1","prob_2","prob_3","prob_4","prob_5","prob_6","prob_7","prob_8","prob_9",
            'rainfall_' + typ + '.fc_year',	'rainfall_' + typ + '.fc_month',	'rainfall_' + typ + '.fc_day',	'rainfall_' + typ + '.fc_hour'
            ]
    if typ == "train":
        columns.append('Unnamed: 0')
    df.drop(columns=columns, inplace=True)
    return df

In [40]:
def preprocessing_pivoting(df, typ):
    df["index"] = df['rainfall_' + typ + ".dh"] // 12
    df["index"].value_counts()
    idx=['rainfall_' + typ + '.stn4contest', 
        'rainfall_' + typ + '.ef_year', 
        'rainfall_' + typ + '.ef_month', 
        'rainfall_' + typ + '.ef_day', 
        'rainfall_' + typ + '.ef_hour',
        'rainfall_' + typ + '.class_interval'
        ]
    if typ=="train":
        idx.append('rainfall_' + typ + '.vv')
    pivot_df = df.pivot_table(index=idx,
                            columns='index',
                            values='expected').reset_index()
    final_df = pivot_df.groupby([
    'rainfall_' + typ + '.stn4contest', 
    'rainfall_' + typ + '.ef_year', 
    'rainfall_' + typ + '.ef_month', 
    'rainfall_' + typ + '.ef_day', 
    'rainfall_' + typ + '.ef_hour']).sum().reset_index()
    return final_df


In [41]:
train_df = pd.read_csv("data/rainfall_train.csv", encoding='utf-8', index_col=False)
train_df = preprocessing_expected(train_df, "train")
train_df = preprocessing_pivoting(train_df, "train")
train_df


index,rainfall_train.stn4contest,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.class_interval,rainfall_train.vv,0,1,2,...,11,12,13,14,15,16,17,18,19,20
0,STN001,A,5,1,12,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,STN001,A,5,1,15,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
2,STN001,A,5,1,18,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,STN001,A,5,1,21,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
4,STN001,A,5,2,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77518,STN020,C,10,10,9,3,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.31
77519,STN020,C,10,10,12,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.31,0.00
77520,STN020,C,10,10,15,4,1.9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.31,0.00
77521,STN020,C,10,10,18,4,1.9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.22,0.00


In [42]:
test_df = pd.read_csv("data/rainfall_test.csv", encoding='utf-8', index_col=False)
test_df = preprocessing_expected(test_df, "test")
test_df = preprocessing_pivoting(test_df, "test")
test_df


index,rainfall_test.stn4contest,rainfall_test.ef_year,rainfall_test.ef_month,rainfall_test.ef_day,rainfall_test.ef_hour,rainfall_test.class_interval,0,1,2,3,...,10,11,12,13,14,15,16,17,18,19
0,STN031,D,5,7,0,-999.0,0.14,0.05,0.45,0.16,...,0.26,0.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,STN031,D,9,28,6,-999.0,0.06,0.06,0.15,0.04,...,1.52,0.78,1.32,1.46,0.57,0.54,0.80,1.05,0.33,0.36
2,STN032,D,7,31,0,-999.0,0.79,0.13,0.64,0.52,...,0.04,0.37,0.46,1.25,0.58,1.04,0.75,1.15,0.93,1.14
3,STN032,D,8,8,12,-999.0,0.14,0.01,0.06,0.10,...,0.06,0.72,0.64,0.55,1.17,0.86,1.11,1.21,1.45,1.98
4,STN032,D,8,8,15,-999.0,0.07,0.16,0.25,0.31,...,0.20,0.46,0.64,0.55,1.17,0.86,1.11,1.21,1.45,1.98
5,STN034,D,5,10,9,-999.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.10,0.28,0.61,0.11,0.37,0.00
6,STN035,D,7,25,15,-999.0,1.92,1.36,1.68,1.93,...,1.26,1.24,1.35,1.40,1.38,1.25,1.28,1.19,1.20,1.20


In [43]:
def my_train_test_split(train_df, test_df):
    X_train = train_df.drop(columns=[
        'rainfall_train.stn4contest',
        'rainfall_train.ef_year',
        'rainfall_train.ef_hour',
        'rainfall_train.ef_day',
        'rainfall_train.vv',
        'rainfall_train.class_interval'
    ])
    display(X_train.columns)
    y_train = train_df[['rainfall_train.class_interval']]

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    X_test = test_df.drop(columns=[
        'rainfall_test.stn4contest',
        'rainfall_test.ef_year',
        'rainfall_test.ef_hour',
        'rainfall_test.ef_day',
        'rainfall_test.class_interval'
    ]) 
    return X_train, y_train, X_val, y_val, X_test

In [45]:
X_train, y_train, X_val, y_val, X_test = my_train_test_split(train_df, test_df)

Index(['rainfall_train.ef_month',                         0,
                               1,                         2,
                               3,                         4,
                               5,                         6,
                               7,                         8,
                               9,                        10,
                              11,                        12,
                              13,                        14,
                              15,                        16,
                              17,                        18,
                              19,                        20],
      dtype='object', name='index')

In [46]:
catboost = CatBoostClassifier(iterations=100, random_seed=42, silent=True)
lgbm = LGBMClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, random_state=42)

catboost.fit(X_train, y_train)
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)

catboost_val_pred = catboost.predict(X_val)
lgbm_val_pred = lgbm.predict(X_val)
xgb_val_pred = xgb.predict(X_val)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5362
[LightGBM] [Info] Number of data points in the train set: 62018, number of used features: 22
[LightGBM] [Info] Start training from score -0.168346
[LightGBM] [Info] Start training from score -4.196775
[LightGBM] [Info] Start training from score -3.753794
[LightGBM] [Info] Start training from score -3.836249
[LightGBM] [Info] Start training from score -3.770450
[LightGBM] [Info] Start training from score -3.551936
[LightGBM] [Info] Start training from score -3.955153
[LightGBM] [Info] Start training from score -4.260956
[LightGBM] [Info] Start training from score -5.215097
[LightGBM] [Info] Start training from score -5.407559


In [48]:
# H, F, M, C 계산 함수
def calculate_csi_components(df):
    H = ((df['estimated_rank'] != 0) & (df['rainfall_train.class_interval'] != 0) & (df['estimated_rank'] == df['rainfall_train.class_interval'])).sum()
    F = ((df['estimated_rank'] != 0) & (df['estimated_rank'] != df['rainfall_train.class_interval'])).sum()
    M = ((df['estimated_rank'] == 0) & (df['rainfall_train.class_interval'] != 0)).sum()
    C = ((df['estimated_rank'] == 0) & (df['rainfall_train.class_interval'] == 0)).sum()
    return H, F, M, C

# CSI 계산 함수
def calculate_csi(H, F, M):
    return H / (H + F + M) if (H + F + M) > 0 else 0


def score(pred, y_test):
    results_df = pd.DataFrame({'estimated_rank': pred, 'rainfall_train.class_interval': np.array(y_test).ravel()})
        # H, F, M, C 계산
    H, F, M, C = calculate_csi_components(results_df)

    # CSI 계산
    csi_value = calculate_csi(H, F, M)
    print(f"CSI: {csi_value}")
    print(f"H: {H}, F: {F}, M: {M}, C: {C}")


In [59]:
display(pd.Series(catboost_val_pred.flatten()).value_counts())
score(catboost_val_pred.flatten(), y_val)

0    14328
5      393
6      254
7      165
4      123
3       84
2       60
8       48
9       41
1        9
Name: count, dtype: int64

CSI: 0.11550759392486011
H: 289, F: 888, M: 1325, C: 13003


In [58]:
display(pd.Series(lgbm_val_pred.flatten()).value_counts())
score(lgbm_val_pred.flatten(), y_val)


0    14350
5      385
6      223
7      163
4      126
3       76
2       57
9       56
8       55
1       14
Name: count, dtype: int64

CSI: 0.11937299035369775
H: 297, F: 858, M: 1333, C: 13017


In [57]:
display(pd.Series(xgb_val_pred.flatten()).value_counts())
score(xgb_val_pred.flatten(), y_val)

0    14284
5      381
6      237
7      183
4      152
3       93
2       73
9       42
8       35
1       25
Name: count, dtype: int64

CSI: 0.11411530815109344
H: 287, F: 934, M: 1294, C: 12990


In [55]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming y_train, y_val, y_test are in integer form
num_classes = len(np.unique(y_train))

# One-hot encode the labels for LSTM
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_val_cat = to_categorical(y_val, num_classes=num_classes)
# y_test_cat = to_categorical(y_test, num_classes=num_classes)

# LSTM을 위한 스케일링
scaler = MinMaxScaler(feature_range=(0, 1))

# 'rainfall_train.ef_month' 컬럼을 제외한 나머지 컬럼 사용
X_train_features = X_train.iloc[:, 1:20].values
X_val_features = X_val.iloc[:, 1:20].values
X_test_features = X_test.iloc[:, 1:20].values

# X_train, X_val, X_test에 대해 정규화 진행
X_train_scaled = scaler.fit_transform(X_train_features)
X_val_scaled = scaler.transform(X_val_features)
X_test_scaled = scaler.transform(X_test_features)

# LSTM 입력 형식으로 변환
timesteps = X_train_scaled.shape[1]

# 3차원 배열로 변환
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, 1))
X_val_lstm = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, 1))

# LSTM 모델 구성
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(timesteps, 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(num_classes, activation='softmax'))  # 다중 클래스 분류를 위해 softmax 활성화 함수 사용

# Adam 옵티마이저 사용
optimizer = Adam(learning_rate=0.001)

lstm_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
lstm_model.fit(X_train_lstm, y_train_cat, epochs=100, batch_size=32, validation_data=(X_val_lstm, y_val_cat))

# LSTM 예측 (클래스 레이블)
lstm_val_pred_proba = lstm_model.predict(X_val_lstm)
lstm_val_pred = np.argmax(lstm_val_pred_proba, axis=1)

2024-06-27 16:21:51.561681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-27 16:21:51.562159: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-27 16:21:51.588004: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


2024-06-27 16:21:53.147061: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


1939/1939 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8414 - loss: 0.6836 - val_accuracy: 0.8481 - val_loss: 0.5542
Epoch 2/100
1939/1939 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8495 - loss: 0.5471 - val_accuracy: 0.8531 - val_loss: 0.5363
Epoch 3/100
1939/1939 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8515 - loss: 0.5400 - val_accuracy: 0.8549 - val_loss: 0.5365
Epoch 4/100
1939/1939 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8561 - loss: 0.5220 - val_accuracy: 0.8553 - val_loss: 0.5216
Epoch 5/100
1939/1939 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8542 - loss: 0.5146 - val_accuracy: 0.8559 - val_loss: 0.5093
Epoch 6/100
1939/1939 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8556 - loss: 0.5079 - val_accuracy: 0.8562 - val_loss: 0.5055
Epoch 7/100
1939/1939 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8560 - loss: 0.5032 - val_accuracy: 0.8551 - val_loss: 0.5034
Epoch 8/100
1939/1939 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8570 - loss: 0.502

In [65]:
display(pd.Series(lstm_val_pred.flatten()).value_counts())
score(lstm_val_pred.flatten(), y_val)


0    14311
5      440
6      234
7      199
4      113
2       67
3       59
9       35
1       27
8       20
Name: count, dtype: int64

CSI: 0.09841897233201581
H: 249, F: 945, M: 1336, C: 12975


In [ ]:
val_preds = {
    "xgb": xgb_val_pred,
    "cat": catboost_val_pred.ravel(),
    "lgbm": lgbm_val_pred,
    "lstm": lstm_val_pred
}
val_df = pd.DataFrame(val_preds)
val_df.to_csv("./data/val.csv", encoding='utf-8', index=False)


In [66]:
from sklearn.model_selection import KFold

In [68]:
# Create a DataFrame with predictions from individual models
stacked_predictions = np.column_stack((catboost_val_pred.ravel(), lgbm_val_pred, xgb_val_pred, lstm_val_pred))
stacked_df = pd.DataFrame(stacked_predictions, columns=['catboost', 'lgbm', 'xgb', 'lstm'])


# Initialize the meta-model (e.g., RandomForestClassifier)
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the meta-model
meta_model.fit(stacked_df, y_val)

# Predict on the validation set
stacked_val_pred = meta_model.predict(X_test)

In [69]:
pd.Series(stacked_val_pred).value_counts()

0    2837
6      67
4      50
5      39
7      36
3      20
9      16
1      13
8      12
2      11
Name: count, dtype: int64

In [70]:
display(pd.Series(stacked_test_pred.flatten()).value_counts())
score(stacked_test_pred.flatten(), y_val)


NameError: name 'stacked_test_pred' is not defined